# House Prices: 包括的なデータ探索（EDA）

このノートブックでは、**Comprehensive Data Exploration with Python**のアプローチを参考に、
初学者向けに分かりやすく包括的なEDA（探索的データ分析）を行います。

## 📚 このノートブックの目的

1. **データの理解**: データの構造、分布、特徴を理解する
2. **問題点の発見**: 欠損値、外れ値、異常なパターンを発見する
3. **特徴量の理解**: 各特徴量が目的変数にどう影響するかを理解する
4. **次のステップの準備**: 特徴量エンジニアリングやモデル構築の方向性を決める

## 🔄 EDAの流れ

```
1. ライブラリのインポート
   ↓
2. データの読み込み
   ↓
3. データの基本情報（形状、型、欠損値）
   ↓
4. 目的変数（SalePrice）の分析
   ↓
5. 数値特徴量の分析
   ↓
6. カテゴリ特徴量の分析
   ↓
7. 相関分析
   ↓
8. 外れ値の検出
   ↓
9. まとめと次のステップ
```


## ステップ1: ライブラリのインポート


In [ ]:
# データ処理
import pandas as pd
import numpy as np

# 可視化
import matplotlib.pyplot as plt
import seaborn as sns

# 統計
from scipy import stats
from scipy.stats import norm, skew

# 警告を非表示
import warnings
warnings.filterwarnings('ignore')

# pandasの表示オプション設定（出力を見やすくする）
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# 日本語フォントの設定（必要に応じて）
plt.rcParams['font.size'] = 12
sns.set_style("whitegrid")

print("✅ ライブラリのインポート完了")


## ステップ2: データの読み込み


In [ ]:
# データの読み込み
try:
    train_df = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
    test_df = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')
    print("✅ ローカル環境からデータを読み込みました")
except FileNotFoundError:
    try:
        train_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
        test_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
        print("✅ Kaggle環境からデータを読み込みました")
    except FileNotFoundError:
        train_df = pd.read_csv('input/house-prices-advanced-regression-techniques/train.csv')
        test_df = pd.read_csv('input/house-prices-advanced-regression-techniques/test.csv')
        print("✅ 別のパスからデータを読み込みました")

print(f"\n訓練データ: {train_df.shape[0]}行 × {train_df.shape[1]}列")
print(f"テストデータ: {test_df.shape[0]}行 × {test_df.shape[1]}列")


## ステップ3: データの基本情報

### 3.1 データの形状と基本情報

まず、データがどのような構造になっているかを確認します。


In [ ]:
# データの基本情報
print("=" * 60)
print("📊 訓練データの基本情報")
print("=" * 60)
train_df.info()

print("\n" + "=" * 60)
print("📊 最初の5行を確認")
print("=" * 60)
train_df.head()


### 3.2 数値特徴量とカテゴリ特徴量の分類

特徴量を数値型とカテゴリ型に分けます。これにより、それぞれに適した分析方法を適用できます。


In [ ]:
# 数値特徴量とカテゴリ特徴量を分離
numeric_cols = train_df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = train_df.select_dtypes(include=['object']).columns.tolist()

# ID列と目的変数を除外
if 'Id' in numeric_cols:
    numeric_cols.remove('Id')
if 'SalePrice' in numeric_cols:
    numeric_cols.remove('SalePrice')

print(f"数値特徴量: {len(numeric_cols)}個")
print(f"カテゴリ特徴量: {len(categorical_cols)}個")
print(f"合計: {len(numeric_cols) + len(categorical_cols)}個")


### 3.3 欠損値の分析

欠損値は、データの品質に大きく影響します。どの特徴量に欠損値があるか、どの程度欠損しているかを確認します。


In [ ]:
# 欠損値の分析
sns.set_style("whitegrid")

missing = train_df.isnull().sum()
missing = missing[missing > 0]
missing.sort_values(inplace=True, ascending=False)

print(f"欠損値がある列: {len(missing)}列")
print("\n欠損値の多い特徴量（上位20個）:")
print(missing.head(20))

# 可視化
if len(missing) > 0:
    plt.figure(figsize=(12, 8))
    missing.head(20).plot.bar()
    plt.title('欠損値の多い特徴量（上位20個）')
    plt.xlabel('特徴量')
    plt.ylabel('欠損数')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()


## ステップ4: 目的変数（SalePrice）の分析

目的変数の分布を理解することは、回帰問題において非常に重要です。
分布の形状によって、前処理（対数変換など）が必要かどうかが決まります。

### 4.1 基本統計量


In [ ]:
target_col = 'SalePrice'

print("=" * 60)
print("📊 目的変数（SalePrice）の基本統計")
print("=" * 60)
# 小数点以下2桁に統一して表示（左揃え）
stats_summary = train_df[target_col].describe().round(2)
# DataFrameに変換してスタイリングで左揃えに設定
stats_df = pd.DataFrame(stats_summary, columns=[target_col])
display(stats_df.style.set_properties(**{'text-align': 'left'}))

print(f"\n💡 歪度（skewness）: {train_df[target_col].skew():.4f}")
print("   （歪度が0に近いほど正規分布に近い）")
print("   （歪度が大きい場合（>0.5）、対数変換を検討）")


### 4.2 分布の可視化


In [ ]:
# 目的変数の分布を可視化
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. ヒストグラム
axes[0, 0].hist(train_df[target_col], bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('住宅価格の分布（ヒストグラム）')
axes[0, 0].set_xlabel('価格（ドル）')
axes[0, 0].set_ylabel('件数')

# 2. 正規分布との比較
axes[0, 1].hist(train_df[target_col], bins=50, density=True, alpha=0.7, label='データ')
x = np.linspace(train_df[target_col].min(), train_df[target_col].max(), 100)
axes[0, 1].plot(x, norm.pdf(x, train_df[target_col].mean(), train_df[target_col].std()), 'r-', label='正規分布')
axes[0, 1].set_title('正規分布との比較')
axes[0, 1].set_xlabel('価格（ドル）')
axes[0, 1].legend()

# 3. 対数正規分布との比較
axes[1, 0].hist(train_df[target_col], bins=50, density=True, alpha=0.7, label='データ')
axes[1, 0].set_title('対数正規分布との比較')
axes[1, 0].set_xlabel('価格（ドル）')
axes[1, 0].legend()

# 4. 対数変換後の分布
y_log = np.log1p(train_df[target_col])
axes[1, 1].hist(y_log, bins=50, edgecolor='black', alpha=0.7)
axes[1, 1].set_title('対数変換後の分布')
axes[1, 1].set_xlabel('log(価格)')
axes[1, 1].set_ylabel('件数')

plt.tight_layout()
plt.show()

# 対数変換後の歪度
print(f"\n💡 対数変換後の歪度: {y_log.skew():.4f}")
print("   対数変換により、分布が正規分布に近づきました")


### 4.3 Q-Qプロット（正規性の検証）

Q-Qプロットは、データが正規分布に従っているかを視覚的に確認する方法です。


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# 変換前のQ-Qプロット
stats.probplot(train_df[target_col], dist="norm", plot=axes[0])
axes[0].set_title('Q-Qプロット（変換前）')

# 対数変換後のQ-Qプロット
stats.probplot(y_log, dist="norm", plot=axes[1])
axes[1].set_title('Q-Qプロット（対数変換後）')

plt.tight_layout()
plt.show()

print("💡 Q-Qプロットで、点が直線上に並んでいれば正規分布に近い")
print("   対数変換により、より正規分布に近づいていることが確認できます")


## ステップ5: 数値特徴量の分析

### 5.1 数値特徴量と目的変数の相関

相関が高い特徴量は、目的変数の予測に重要です。


In [ ]:
# 相関を計算
correlations = train_df[numeric_cols + [target_col]].corr()[target_col].sort_values(ascending=False)
correlations = correlations.drop(target_col)  # 自分自身との相関を除外

print("=" * 60)
print("📊 数値特徴量と目的変数の相関（上位20個）")
print("=" * 60)
print("\n正の相関が高い特徴量（上位10個）:")
print(correlations.head(10))

print("\n負の相関が高い特徴量（下位10個）:")
print(correlations.tail(10))

# 可視化
plt.figure(figsize=(10, 12))
correlations.head(20).plot(kind='barh')
plt.title('数値特徴量と目的変数の相関（上位20個）')
plt.xlabel('相関係数')
plt.tight_layout()
plt.show()

# 相関が強い特徴量を保存
high_corr_features = correlations[abs(correlations) > 0.3].index.tolist()
print(f"\n💡 相関が強い特徴量（|相関| > 0.3）: {len(high_corr_features)}個")
print(f"   これらの特徴量は価格予測に重要です")


### 5.2 重要な数値特徴量の詳細分析

相関が高い特徴量について、より詳しく分析します。


In [ ]:
# 相関が高い上位5個の特徴量を選択
top_features = correlations.head(5).index.tolist()

# 可視化
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

# 目的変数の分布
axes[0].hist(train_df[target_col], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_title(f'{target_col}の分布')
axes[0].set_xlabel('価格（ドル）')

# 上位5個の特徴量と目的変数の散布図
for i, feature in enumerate(top_features[:5], 1):
    axes[i].scatter(train_df[feature], train_df[target_col], alpha=0.3)
    axes[i].set_xlabel(feature)
    axes[i].set_ylabel(target_col)
    axes[i].set_title(f'{feature} vs {target_col}\n(相関: {correlations[feature]:.3f})')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


### 5.3 数値特徴量の分布（歪度の確認）

数値特徴量の分布が正規分布から大きく外れている場合、変換が必要かもしれません。


In [ ]:
# 数値特徴量の歪度を計算
skewed_features = train_df[numeric_cols].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)

print("=" * 60)
print("📊 数値特徴量の歪度（上位10個）")
print("=" * 60)
print(skewed_features.head(10))

# 歪度が大きい特徴量（|歪度| > 0.75）を可視化
high_skew_features = skewed_features[abs(skewed_features) > 0.75].index.tolist()

if len(high_skew_features) > 0:
    print(f"\n💡 歪度が大きい特徴量（|歪度| > 0.75）: {len(high_skew_features)}個")
    
    # 可視化（最初の6個）
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    axes = axes.flatten()
    
    for i, feature in enumerate(high_skew_features[:6]):
        axes[i].hist(train_df[feature].dropna(), bins=50, edgecolor='black', alpha=0.7)
        axes[i].set_title(f'{feature}\n(歪度: {skewed_features[feature]:.2f})')
        axes[i].set_xlabel(feature)
        axes[i].set_ylabel('件数')
    
    plt.tight_layout()
    plt.show()
    
    print("💡 これらの特徴量は、対数変換やBox-Cox変換を検討してください")


## ステップ6: カテゴリ特徴量の分析

### 6.1 カテゴリ特徴量と目的変数の関係

カテゴリ特徴量が目的変数にどのように影響するかを確認します。


In [ ]:
# カテゴリ特徴量と目的変数の関係を分析
cat_stats = []

for col in categorical_cols[:10]:  # 最初の10個を分析
    stats = train_df.groupby(col)[target_col].agg(['count', 'mean', 'std']).sort_values('mean')
    stats['cv'] = stats['std'] / stats['mean']  # 変動係数
    
    cat_stats.append({
        '特徴量': col,
        'カテゴリ数': train_df[col].nunique(),
        '平均価格の範囲': stats['mean'].max() - stats['mean'].min(),
        '最小平均価格': stats['mean'].min(),
        '最大平均価格': stats['mean'].max()
    })

cat_stats_df = pd.DataFrame(cat_stats)
print("=" * 60)
print("📊 カテゴリ特徴量の統計（最初の10個）")
print("=" * 60)
print(cat_stats_df)

# 最も価格差が大きいカテゴリ特徴量を可視化
if len(cat_stats_df) > 0:
    best_cat = cat_stats_df.loc[cat_stats_df['平均価格の範囲'].idxmax(), '特徴量']
    print(f"\n💡 価格差が最も大きいカテゴリ特徴量: {best_cat}")
    
    # 可視化
    plt.figure(figsize=(12, 6))
    train_df.boxplot(column=target_col, by=best_cat, ax=plt.gca())
    plt.title(f'{best_cat} と 住宅価格の関係')
    plt.suptitle('')  # デフォルトのタイトルを削除
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()


## ステップ7: 相関分析

### 7.1 相関行列（ヒートマップ）

特徴量間の相関を確認することで、多重共線性（特徴量同士が強く相関している）を発見できます。


In [ ]:
# 相関が高い特徴量を選択（計算を高速化するため）
top_corr_features = correlations.head(10).index.tolist() + [target_col]

# 相関行列を計算
corr_matrix = train_df[top_corr_features].corr()

# ヒートマップで可視化
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('特徴量間の相関行列（重要な特徴量のみ）')
plt.tight_layout()
plt.show()

print("💡 相関が高い（|相関| > 0.8）特徴量のペアは、多重共線性の可能性があります")
print("   そのような場合、一方の特徴量を削除することを検討してください")


## ステップ8: 外れ値の検出

### 8.1 目的変数の外れ値

外れ値は、モデルの性能に大きな影響を与える可能性があります。


In [ ]:
# 外れ値を検出（IQR法）
Q1 = train_df[target_col].quantile(0.25)
Q3 = train_df[target_col].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = train_df[(train_df[target_col] < lower_bound) | (train_df[target_col] > upper_bound)]

print(f"外れ値の定義:")
print(f"  下限: ${lower_bound:,.2f}")
print(f"  上限: ${upper_bound:,.2f}")
print(f"  外れ値の数: {len(outliers)} ({len(outliers)/len(train_df)*100:.2f}%)")

# 可視化
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# 箱ひげ図
axes[0].boxplot(train_df[target_col])
axes[0].set_title('住宅価格の箱ひげ図（外れ値検出）')
axes[0].set_ylabel('価格（ドル）')

# 散布図（ID vs 価格）
axes[1].scatter(train_df.index, train_df[target_col], alpha=0.5)
axes[1].axhline(y=upper_bound, color='r', linestyle='--', label=f'上限: ${upper_bound:,.0f}')
axes[1].axhline(y=lower_bound, color='r', linestyle='--', label=f'下限: ${lower_bound:,.0f}')
axes[1].set_title('住宅価格の散布図（外れ値検出）')
axes[1].set_xlabel('サンプル番号')
axes[1].set_ylabel('価格（ドル）')
axes[1].legend()

plt.tight_layout()
plt.show()

if len(outliers) > 0:
    print(f"\n⚠️ 外れ値の統計情報:")
    print(outliers[target_col].describe())


### 8.2 重要な特徴量と目的変数の外れ値

相関が高い特徴量と目的変数の関係で、外れ値となるサンプルを確認します。


In [ ]:
# 相関が高い特徴量と目的変数の散布図で外れ値を確認
if len(top_features) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # 最初の2つの特徴量を可視化
    for i, feature in enumerate(top_features[:2]):
        axes[i].scatter(train_df[feature], train_df[target_col], alpha=0.5)
        axes[i].set_xlabel(feature)
        axes[i].set_ylabel(target_col)
        axes[i].set_title(f'{feature} vs {target_col}')
        axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("💡 散布図で、他の点から大きく外れている点が外れ値の可能性があります")
